## Validate ARGO R4 Resources from Servers

- Get Resources
- Load into R4 IG
- Run IG 
- Review QA report for Examples

(Python 3.7) 
**Run in MAC local source directory**

### Import modules and asign globals

In [1]:
import os #os module imported here
from json import load, dumps, loads
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from pathlib import Path
from pandas import *
from collections import namedtuple

#Globals

# if working on Mac '/Users/ehaas/...'
# if working on PC '//ERICS-AIR-2/ehaas...'
source_path = '/ehaas/Documents/FHIR/USCoreR4Validator'
examples_path = '/ehaas/Documents/FHIR/USCoreR4Validator/source/examples'

headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

# US Core R4 Profiles
profiles ={
'CareTeam': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam',
'MedicationStatement': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationstatement',
'Device': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-device',
'Practitioner': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner',
'Patient': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient',
'Immunization': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-immunization',
'Observation1': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-smokingstatus',
'DocumentReference': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference',
'MedicationRequest': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest',
'Condition': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition',
'Encounter': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-encounter',
'Organization': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization',
'Observation2': 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-weight-for-height',
'Procedure': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-procedure',
'Medication': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-medication',
'Observation3': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab',
'DiagnosticReport1': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note',
'PractitionerRole': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole',
'CarePlan': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan',
'Goal': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-goal',
'Location': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-location',
'Observation4': 'http://hl7.org/fhir/us/core/StructureDefinition/pediatric-bmi-for-age',
'DiagnosticReport2': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-lab',
'AllergyIntolerance': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance',
}

f_servers = dict(epic='https://connectathon.epic.com/Interconnect-Fhir-Unsecure/api/FHIR/R4/')

### Optionally Clear example folder first...  (todo)

### Get examples from FHIR Server

- import data from spreadsheet
- convert to pandas df and access using dot notation


In [2]:
server = 'epic'  # sheet name = server
xls = ExcelFile('test_resources.xlsx')
df = read_excel(xls,server,na_false = False)

df

,resource_type,id
0,Patient,eroCd17NbCffM3E3WXi7eOQ3
1,Practitioner,e7D.u41Zv3VPrLmUp-mWnrQ3
2,Organization,enRyWnSP963FYDpoks4NHOA3
3,AllergyIntolerance,eyHIVsZwhzaZhX9XvFesx5A3
4,Encounter,"ehoN2kZnYuW5z0Di4Kkno5w3,eiSX3g2gt4tJIs7-6h7nJ..."
5,Location,e4W4rmGe9QzuGm2Dy4NBqVc0KDe6yGld6HW95UuN-Qd03


### Add meta profile to example

- for each resource type fetching instance
  - instance are comma separated list of ids
- convert json to dict
- insert profile
- update all references to relative references using same server ids
- convert back to json

- save as example in examples folder in ig publisher 
- create a df file and export as csv data file for mapping the file name to id for ig publisher

In [4]:
def make_rel_ref(dictionary): # replace all absolute references with relative references
    for k, v in dictionary.items():
        if k == 'reference':    
            
            # make rel ref assuming is Type/id
            ref_path = dictionary[k].split('/')
            dictionary[k] = f'{ref_path[-2]}/{ref_path[-1]}'
        elif isinstance(v, dict):
            make_rel_ref(v)
        elif isinstance(v, list):
            for d in v:
                if isinstance(d, dict):
                    make_rel_ref(d)
    return(dictionary)

data_file = {'file_name': [],
        'id': [],
        }
for i in df.itertuples(index=True):
    for j in i.id.split(','):
        print(i.resource_type,j)
        params = dict(
            _id = j
            )
        # Open Server
        r = get(f'{f_servers[server]}{i.resource_type}/{j}', headers = headers)
        fhir_d = r.json() if r.status_code==200 else None
        print(r.status_code)

        if r.status_code!=200:
            print(f'Fetching of {f_servers[server]}{i.resource_type}/{j} failed: Status code= {r.status_code}')
            r = get(f'{f_servers[server]}{i.resource_type}',params = params, headers = headers ) # try other syntax

            fhir_d = r.json()['entry'][0]['resource'] if r.status_code==200 else None #unbundle
            # wait for error to do try pattern
            print(r.status_code)
        if r.status_code!=200:
            print(f'Seaching using {f_servers[server]}{i.resource_type}?_id={j} failed: Status code= {r.status_code}')     
        else: # r.status_code==200
            try:
                fhir_d['meta']['profile'].append(profiles[i.resource_type])
            except KeyError:
                fhir_d['meta']= dict(profile = [profiles[i.resource_type]])
            fhir_d = make_rel_ref(fhir_d)  # make rel ref assuming is Type/id
            fhir_json = dumps(fhir_d, indent = 4)
            print(f'{"*"*10} {server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}.json {"*"*10}')
            print(fhir_json)
            try:
                p = Path(f'/Users{examples_path}') / f'{server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}.json'
                #print(p)
                p.write_text(fhir_json, encoding='utf-8')
                print('writing to examples folder......')
            except FileNotFoundError: # try different path
                p = Path(f'//ERICS-AIR-2{examples_path}') / f'{server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}.json'
                #print(p)
                p.write_text(fhir_json, encoding='utf-8')
                print('writing to examples folder......')        
            data_file['file_name'].append(f'{server}-{fhir_d["resourceType"].lower()}-{fhir_d["id"]}')
            data_file['id'].append(fhir_d["id"])
  

Patient eroCd17NbCffM3E3WXi7eOQ3
200
********** epic-patient-eroCd17NbCffM3E3WXi7eOQ3.json **********
{
    "resourceType": "Patient",
    "id": "eroCd17NbCffM3E3WXi7eOQ3",
    "active": true,
    "gender": "male",
    "birthDate": "1978-12-11",
    "deceasedBoolean": false,
    "extension": [
        {
            "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex",
            "valueCode": "M"
        },
        {
            "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-race",
            "extension": [
                {
                    "url": "ombCategory",
                    "valueCoding": {
                        "system": "http://hl7.org/fhir/us/core/ValueSet/omb-race-category",
                        "code": "2028-9",
                        "display": "Asian"
                    }
                },
                {
                    "url": "text",
                    "valueString": "Asian"
                }
            ]
     

FileNotFoundError: [Errno 2] No such file or directory: '\\Users\\ehaas\\Documents\\FHIR\\USCoreR4Validator\\source\\examples\\epic-patient-eroCd17NbCffM3E3WXi7eOQ3.json'

#### Save file name, id data as csv file

In [4]:
df_data = DataFrame(data_file, columns= ['file_name', 'id'])
export_csv = df_data.to_csv(f'{source_path}/framework/_data/examples.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
print('saving csv .......')

saving csv .......


### Validate using IG Build

1. run py script to update ig.json and ig.xml


In [5]:
%%bash
set -e
cd /Users/ehaas/Documents/FHIR/USCoreR4Validator/
path1=/Users/ehaas/Downloads/org.hl7.fhir.igpublisher.jar
source_path=/Users/ehaas/Documents/FHIR/USCoreR4Validator
path3=/Users/ehaas/Documents/FHIR/IG-tools/
echo "================================================================="
echo === use definition files from relative path ../$SOURCE ===
echo "================================================================="
echo getting rid of .DS_Store files since they gum up the igpublisher....
echo "================================================================="
echo === run definitions maker with optional source directory name as first argument ===
echo === create ig.json and ig.xml in $PWD and ../$SOURCE ===
echo "================================================================="
find . -name '.DS_Store' -type f -delete
python3.5 ${path3}definitions.py

=== use definition files from relative path ../ ===
getting rid of .DS_Store files since they gum up the igpublisher....
=== run definitions maker with optional source directory name as first argument ===
=== create ig.json and ig.xml in /Users/ehaas/Documents/FHIR/USCoreR4Validator and ../ ===


 2019-06-19 09:17:00,350 - INFO- Start of program
 2019-06-19 09:17:00,350 - INFO- The logging module is working.
 2019-06-19 09:17:00,351 - INFO- create the ig.json file template as dictionary
 2019-06-19 09:17:00,351 - INFO- root dir = /Users/ehaas/Documents/FHIR/USCoreR4Validator/
 2019-06-19 09:17:00,351 - INFO- relative address of source_dir = 
 2019-06-19 09:17:00,351 - INFO- definitions.csv file is at: definitions.csv
 2019-06-19 09:17:00,354 - INFO- k,v = allviews, False
 2019-06-19 09:17:00,354 - INFO- updating ig.json with this:  { "allviews": "False" }
 2019-06-19 09:17:00,354 - INFO- k,v = broken-links, warning
 2019-06-19 09:17:00,354 - INFO- updating ig.json with this:  { "broken-links": "warning" }
 2019-06-19 09:17:00,354 - INFO- k,v = canonicalBase, http://hl7.org/fhir/us/core
 2019-06-19 09:17:00,354 - INFO- updating ig.json with this:  { "canonicalBase": "http://hl7.org/fhir/us/core" }
 2019-06-19 09:17:00,354 - INFO- k,v = codesystems, 
 2019-06-19 09:17:00,354 - IN

### Strip out extra stuff stuff from ig.json to make run faster

In [9]:
remove_default_list = [
                        'StructureDefinition',
                        'CodeSystem',
                        'ValueSet',
                        'OperationDefinition',
                        'ConceptMap',
                        'CapabilityStatement',
                        'SearchParameter',
                        'StructureMap',
                       ] 
p = Path(r'/Users/ehaas/Documents/FHIR/USCoreR4Validator/ig.json')
ig_dict = loads(p.read_text())
ig_defaults = ig_dict['defaults']
# Using items() + dict comprehension to remove a dict. pair 
# removes  remove_defaults_list
new_ig_defaults = {key:val for key, val in ig_defaults.items() if key not in remove_default_list} 
ig_dict['defaults'] = new_ig_defaults
p.write_text(dumps(ig_dict,indent=4))

35011

### Run the ig publisher locally ( until can run and see on github )

1. run ig publisher to validate examples **it will take a couple of minutes to run** ( until I can use a more stripped down template )

In [7]:
%%bash
set -e
cd /Users/ehaas/Documents/FHIR/USCoreR4Validator/
path1=/Users/ehaas/Downloads/org.hl7.fhir.igpublisher.jar
java -jar ${path1} -ig ig.json

FHIR Implementation Guide Publisher Version 0.9.36-SNAPSHOT - Built 2019-06-19T10:03:38.413+10:00 - Git 74eb9556a74f
Detected Java version: 10.0.2 from /Library/Java/JavaVirtualMachines/jdk-10.0.2.jdk/Contents/Home on x86_64 (64bit). 2048MB available
Run time = Wednesday, June 19, 2019 at 9:17:03 AM Pacific Daylight Time
[/Users/ehaas/Documents/FHIR/USCoreR4Validator] -ig ig.json
Package Cache: /Users/ehaas/.fhir/packages
Load Configuration from /Users/ehaas/Documents/FHIR/USCoreR4Validator/ig.json    (00.0311sec)
Logging progress
Contacting Build Server...                                                       (00.0361sec)
 ... done                                                                        (02.0580sec)
Root directory: /Users/ehaas/Documents/FHIR/USCoreR4Validator                    (02.0582sec)
Terminology Cache is at /Users/ehaas/Documents/FHIR/USCoreR4Validator/generated_output/txCache. 33 files in cache (02.0830sec)
Load hl7.fhir.core-4.0.0 package from /Users/ehaas/.fh

### Disabled code .... Get qa.html output and display
(or alternatively go here: consider using autopublish to share)

### Commit to GitHub 
- for now can see the validation of examples here: https://healthedata1.github.io/USCoreR4Validator/all-examples.html
- In future Will trigger autobuilder and a CI build

In [8]:
!git -C /Users/ehaas/Documents/FHIR/USCoreR4Validator add .
!git -C /Users/ehaas/Documents/FHIR/USCoreR4Validator commit -m "added new examples to validate"
!git -C /Users/ehaas/Documents/FHIR/USCoreR4Validator push

[master 734f0f6e2] added new examples to validate
 543 files changed, 23657 insertions(+), 5797 deletions(-)
 create mode 100644 docs/Location-e4W4rmGe9QzuGm2Dy4NBqVc0KDe6yGld6HW95UuN-Qd03.html
 create mode 100644 docs/Location-e4W4rmGe9QzuGm2Dy4NBqVc0KDe6yGld6HW95UuN-Qd03.json
 create mode 100644 docs/Location-e4W4rmGe9QzuGm2Dy4NBqVc0KDe6yGld6HW95UuN-Qd03.json.html
 create mode 100644 docs/Location-e4W4rmGe9QzuGm2Dy4NBqVc0KDe6yGld6HW95UuN-Qd03.xml
 create mode 100644 docs/Organization-enRyWnSP963FYDpoks4NHOA3.html
 create mode 100644 docs/Organization-enRyWnSP963FYDpoks4NHOA3.json
 create mode 100644 docs/Organization-enRyWnSP963FYDpoks4NHOA3.json.html
 create mode 100644 docs/Organization-enRyWnSP963FYDpoks4NHOA3.xml
 rewrite docs/package.tgz (93%)
 create mode 100644 docs/usage-stats.json
 create mode 100644 source/examples/epic-location-e4W4rmGe9QzuGm2Dy4NBqVc0KDe6yGld6HW95UuN-Qd03.json
 create mode 100644 source/examples/epic-organization-enRyWnSP963FYDpoks4NHOA3.json
 rewrite sou